## Build base Docker image (if not already built)

In [ ]:
%%bash
docker build -t mlflow-docker-example -f bike-rental-regression/Dockerfile . 

## Train a model

In [ ]:
%%bash
mlflow run bike-rental-regression -P train_year=2011 -P train_month=1 -P test_year=2011 -P test_month=3 -P dataset_path=/data/train_bike.csv

## Package the model in a Docker image and register it in the local registry

In [ ]:
%%bash
mlflow models build-docker --model-uri runs:/c258b637e0a64ffe979248f475d4c94b/model --name "my-image-name"

## Tag the Docker image in order to push it to GCP Artifact Registry

In [ ]:
%%bash
docker tag my-image-name:latest europe-north1-docker.pkg.dev/aalto-atss/cyclone-docker-repo/my-image-name:latest

## Push the image

In [18]:
%%bash
docker push europe-north1-docker.pkg.dev/aalto-atss/cyclone-docker-repo/my-image-name:latest

The push refers to repository [europe-north1-docker.pkg.dev/aalto-atss/cyclone-docker-repo/my-image-name]
ba3832347bc7: Preparing
1e5fb491959e: Preparing
439d36d2da31: Preparing
d0cdc2725501: Preparing
9a421c24b37a: Preparing
6719756ec0b7: Preparing
931f8fb95381: Preparing
22843a8b08ac: Preparing
4554b763a583: Preparing
c38454c68f66: Preparing
2fb7b764853b: Preparing
824bf068fd3d: Preparing
6719756ec0b7: Waiting
931f8fb95381: Waiting
22843a8b08ac: Waiting
4554b763a583: Waiting
c38454c68f66: Waiting
824bf068fd3d: Waiting
2fb7b764853b: Waiting
d0cdc2725501: Layer already exists
1e5fb491959e: Layer already exists
ba3832347bc7: Layer already exists
9a421c24b37a: Layer already exists
439d36d2da31: Layer already exists
6719756ec0b7: Layer already exists
931f8fb95381: Layer already exists
22843a8b08ac: Layer already exists
4554b763a583: Layer already exists
c38454c68f66: Layer already exists
2fb7b764853b: Layer already exists
824bf068fd3d: Layer already exists
latest: digest: sha256:6e2557773

## Deploy on GCR

Create a new service on GCR, setting the port to 8000, setting `DISABLE_NGINX` to `true`, and `GUNICORN_CMD_ARGS` to `--bind=0.0.0.0`.
Essentially we want to replicate on GCR this local command:
```
docker run -p 5001:8000 -e DISABLE_NGINX=true -e GUNICORN_CMD_ARGS="--bind=0.0.0.0" my-image-name:latest
```

## Test that the model deployed on GCR has the expected performance

We get the service's `URL` from GCR's console.

In [17]:
import requests
import json
import pandas as pd
import numpy as np

URL = 'https://my-image-name-ae5g3kdcea-lz.a.run.app/invocations'

dataset = pd.read_csv('data/train_bike.csv', header=0,  sep=',',
                      parse_dates=['datetime'], index_col='datetime').drop(
    columns=['casual', 'registered']
)
test_year = 2011
test_month = 3
test: pd.DataFrame = dataset.loc[
    lambda df: (df.index.year == test_year) & (df.index.month == test_month)
]
def time_features(input_df):
    return input_df.assign(
        month = lambda df: df.index.month,
        day = lambda df: df.index.day,
        hour = lambda df: df.index.hour,
    )
test.pipe(time_features).drop(columns='count').head().to_json(orient='split', index=False)

predictions = requests.post(
    URL,
    json=json.loads(
        test.pipe(time_features)
        .drop(columns='count')
        .to_json(orient='split', index=False)
    )
).json()

from sklearn.metrics import r2_score

assert test_month == 3
assert np.abs(r2_score(test.pipe(time_features)['count'].values, predictions) - 0.603) < 1e-2